In [1]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy

from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
import src.tracking as tracking

from dipy.tracking.streamline import values_from_volume
import dipy.align.vector_fields as vfu
from dipy.core.sphere import Sphere
from dipy.core import subdivide_octahedron 
from scipy.spatial import KDTree
from dipy.core import subdivide_octahedron 

from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.tracking import metrics
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask

import numpy as np
import warnings

import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from keras.models import load_model
from keras.layers import Activation

import importlib
importlib.reload(tracking)
importlib.reload(nn_helper)
import src.tracking as tracking
import tensorflow as tf
from src.nn_helper import swish, squared_cosine_proximity_2

Using TensorFlow backend.
/home/nico/Code/deepFibreTracking/src/tracking.py:128: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(streamlines_left) == len(streamlines_right), "The two lists of streamlines need to have the same number of elements.")


Load seed mask and prepare our data

In [ ]:
sh_order = 4
b_value = 1000
stepWidth = 0.6

In [4]:
coordinateScaling = 1
useDTIPeakDirection = True
useSphericalHarmonics = True
print('Loaded model with  (dx %d, dy %d, dz %d) and %d channels' % (noX, noY, noZ, noC))

# load DWI data
print('Importing HCP data')
bvals,bvecs,gtab,dwi,aff,t1,binarymask = dwi_tools.loadHCPData('data/100307')
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(b_value, bvals, bvecs, dwi)
b0_idx = bvals < 10
b0 = dwi[..., b0_idx].mean(axis=3)

dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)

tracking_data = dwi_singleShell

### exract the averaged b0.

if(useSphericalHarmonics):
    print('Spherical Harmonics (ours)')
    dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)
    tracking_data = dwi_tools.get_spherical_harmonics_coefficients(bvals=bvals_singleShell,bvecs=bvecs_singleShell,sh_order=sh_order, dwi=dwi_singleShell_norm, b0 = 0)
    
    #CSD/DIPY APPROACH
    print('Spherical Harmonics (dipy)')
    #csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, None, sh_order=sh_order)
    #csd_fit = csd_model.fit(dwi_singleShell_norm, mask=binarymask)
    #tracking_data = csd_fit.shm_coeff


# DTI PEAK DIRECTION INPUT
if(useDTIPeakDirection):
    print('DTI Peak Direction/odf estimation')
    import dipy.reconst.dti as dti
    start_time = time.time()
    dti_model = dti.TensorModel(gtab_singleShell, fit_method='LS')
    dti_fit = dti_model.fit(dwi_singleShell)
    dti_fit_odf = dti_fit.odf(sphere = default_sphere)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')
    
# corpus callosum seed mask
ccmask, options = nrrd.read('data/100307/100307-ccSegmentation.nrrd')
ccseeds = seeds_from_mask(ccmask, affine=aff)
# whole brain seeds
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)

Loaded model with  (dx 1, dy 1, dz 1) and 15 channels
Importing HCP data
Spherical Harmonics (ours)
Percentage erroneous voxels: 3.87


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:106: RuntimeWarning: divide by zero encountered in true_divide
  weights_normed = (weights / b0)
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:106: RuntimeWarning: invalid value encountered in true_divide
  weights_normed = (weights / b0)


Spherical Harmonics (dipy)
Percentage erroneous voxels: 3.87
DTI Peak Direction/odf estimation
Runtime 327.906329870224s


# Multi (DWI,PrevTension) -> Tension tracker

In [2]:
pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_doubleIn_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_4172--0.974801.h5'
pModel = 'results/train_prediction_grid_normalized_dti_cs1_wholebrain/models/mlp_doubleIn_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_0_pt_0_4846--0.977051.h5'
pModel = 'results/train_OLDsh4_step0.6_wholeBrain_b1k_csd_1x1x1/models/doubleIn_noInputRepetition_mlp_doubleIn_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_467--0.977036.h5'
pResult = pModel.replace('.h5','').replace('/models/','_') + '-prediction'

In [9]:
tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'squared_cosine_proximity': squared_cosine_proximity_2, 'squared_cosine_proximity_2': squared_cosine_proximity_2})
noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)[0]
#tracker.summary()
useBitracker = True

In [ ]:
from dipy.tracking.streamline import set_number_of_points, Streamlines
sl_est_2 = set_number_of_points(Streamlines(streamlines_joined_imageCS), nb_points = 20)
sl_val_2 = set_number_of_points(Streamlines(streamlines_val.tolist()), nb_points = 20)

In [ ]:
from dipy.segment.metric import SumPointwiseEuclideanMetric
from dipy.segment import quickbundles

dist = quickbundles.bundles_distances_mdf(sl_est_2, sl_val_2)

In [ ]:
dist_minIdx = np.argmin(dist, axis = 1)

---

# Single DWI -> TENSION tracker

In [7]:
#pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_mse_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_950-0.489318.h5'

#pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_cos_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_1173--0.263056.h5'

pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_7217--0.968468.h5'

pModel = 'results/train_OLDsh4_step0.1_wholeBrain_b1k_1x1x1_step10/models/mlp_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_6691--0.990052.h5'

#pModel = 'results/train_prediction_grid_normalized_dti_cs1_wholebrain/models/mlp_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_0_pt_0_9960--0.858613.h5'
pResult = pModel.replace('.h5','').replace('/models/','_') + '-prediction'

In [ ]:
tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'squared_cosine_proximity': squared_cosine_proximity_2, 'squared_cosine_proximity_2': squared_cosine_proximity_2})
noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)
#tracker.summary()
useBitracker = False

# Start Tracking 

Tracking without stopping conditions

In [ ]:
# do tracking (wholebrainseeds, ccseeds)
importlib.reload(tracking)
import src.tracking as tracking
start_time = time.time()
streamlines_mlp_simple, vec_norm = tracking.start(bitracker=useBitracker,inverseDirection=False,seeds=ccseeds,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

start_time = time.time()
streamlines_mlp_simple_2, vec_norm_2 = tracking.start(bitracker=useBitracker,inverseDirection=True,seeds=ccseeds,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

# visualize results
streamlines_mlp_simple_f = np.fliplr(streamlines_mlp_simple)
streamlines_joined = np.concatenate([streamlines_mlp_simple_f,streamlines_mlp_simple_2],axis=1)    

streamlines_joined_imageCS = transform_streamlines(streamlines_joined, np.linalg.inv(aff))
                                                   
#np.save(pResult + '-ijk.vtk', streamlines_joined_imageCS)
#np.save(pResult + '-ras.vtk', streamlines_joined)
dwi_tools.saveVTKstreamlines(streamlines_joined,pResult + '.vtk')
                                                   
#dwi_tools.visStreamlines(streamlines_joined_imageCS,t1, vol_slice_idx = 75)

---
Tracking with stopping conditions

In [19]:
importlib.reload(tracking)
import src.tracking as tracking

start_time = time.time()
streamlines_mlp_simple_sc,vNorms = tracking.startWithStopping(mask=binarymask,fa=dti_fit.fa, bitracker=useBitracker, inverseDirection=False, seeds=wholebrainseeds, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

start_time = time.time()
streamlines_mlp_simple_sc_2,vNorms2 = tracking.startWithStopping(mask=binarymask,fa=dti_fit.fa, bitracker=useBitracker, inverseDirection=True, seeds=wholebrainseeds, data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

# join left and right streamlines
streamlines_joined_sc = tracking.joinTwoAlignedStreamlineLists(streamlines_mlp_simple_sc,streamlines_mlp_simple_sc_2)

streamlines_joined_sc_imageCS = transform_streamlines(streamlines_joined_sc, np.linalg.inv(aff))

dwi_tools.saveVTKstreamlines(streamlines_joined_sc_imageCS,pResult + '.vtk')

/home/nico/Code/deepFibreTracking/src/tracking.py:185: RuntimeWarning: divide by zero encountered in true_divide
  lastDirections = np.nan_to_num(lastDirections / vecNorms[:,None])
/home/nico/Code/deepFibreTracking/src/tracking.py:185: RuntimeWarning: invalid value encountered in true_divide
  lastDirections = np.nan_to_num(lastDirections / vecNorms[:,None])
/home/nico/Code/deepFibreTracking/src/tracking.py:206: RuntimeWarning: invalid value encountered in double_scalars
  theta = np.arcsin(np.dot(pv1,lv1) / (np.linalg.norm(pv1)*np.linalg.norm(lv1)))
/home/nico/Code/deepFibreTracking/src/tracking.py:206: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(np.dot(pv1,lv1) / (np.linalg.norm(pv1)*np.linalg.norm(lv1)))


Runtime 40854.154591321945 s 
Runtime 44543.26001191139 s 


NameError: name 'ddwi_tools' is not defined

# Visualize streamlines

In [14]:
dwi_tools.visStreamlines(streamlines_joined_sc_imageCS,t1, vol_slice_idx = 75)

/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
/home/nico/anaconda3/lib/python3.6/site-packages/dipy/viz/colormap.py:233: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))


In [17]:
dwi_tools.visTwoSetsOfStreamlines(streamlines_joined_imageCS,streamlines_joined_sc_imageCS, t1, vol_slice_idx = 75)
# first set of streamlines is red
# second set of streamline is green

/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
